<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語文章の分類

日本語のデータセットでBERTのモデルをファインチューニングし、ニュースの分類を行います。

## 大きな流れ
①ディレクトリにあるテキストファイルをidと本文をセットにしてリスト化  <br>
②上記を訓練データとテストデータに分割してCSV化
③

## ライブラリのインストール
ライブラリTransformers、およびnlpをインストールします。  

In [1]:
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic
!pip install sentencepiece
!pip install accelerate

     ---------------------------------------- 0.0/126.8 kB ? eta -:--:--
     ------------------------------------ - 122.9/126.8 kB 7.5 MB/s eta 0:00:01
     -------------------------------------- 126.8/126.8 kB 1.5 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp38-cp38-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.0 MB/s eta 0:00:00
  Using cached charset_normalizer-3.3.2-cp38-cp38-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
   ---------------------------------------- 0.0/8.2 MB


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     --------- ------------------------------ 0.4/1.7 MB 8.1 MB/s eta 0:00:01
     -------------- ------------------------- 0.6/1.7 MB 6.4 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 7.1 MB/s eta 0:00:01
     ------------------------------- -------- 1.3/1.7 MB 7.6 MB/s eta 0:00:01
     -------------------------------------- - 1.6/1.7 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 6.6 MB/s eta 0


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/507.1 kB ? eta -:--:--
   ----------------------------------- --- 460.8/507.1 kB 14.5 MB/s eta 0:00:01
   ---------------------------------------- 507.1/507.1 kB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   ---------------------------------------- 166.4/166.4 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/367.1 kB ? eta -:--:--
   --------------------------------------  358.4/367.1 kB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 367.1/367.1 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/132.6 kB ? eta -:--:--
   ---------------------------------------- 132.6/132.6 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
   ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.8 kB ? eta -:--:--
   ---------------

...


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/977.6 kB ? eta -:--:--
     --------------- ---------------------- 399.4/977.6 kB 8.3 MB/s eta 0:00:01
     ------------------------- ------------ 665.6/977.6 kB 7.0 MB/s eta 0:00:01
     ------------------------------------ - 931.8/977.6 kB 6.5 MB/s eta 0:00:01
     -------------------------------------  972.8/977.6 kB 6.8 MB/s eta 0:00:01
     -------------------------------------- 977.6/977.6 kB 4.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     -- ------------------------------------- 0.3/5.7 MB 7.2 MB/s eta 0:00:01
     ----- ---------------------------------- 0.7/5.7 MB 6.6 MB/s eta 0:00:01
     ------- -------------------------------- 1.0/5.7 MB 7.3 MB/s eta 0:00:01
     --------- ------------------------------ 1.3/5.7 MB 7.0 MB/s eta 0:00:01
     ----------- ---------------------------- 1.7/5.7 MB 6.7 MB/s eta 0:00:01
     -------------- ------------------------- 2.1/5.7 MB 6.8 MB/s eta 0:00:01
     ----------------- ---------------------- 2.5/5.7 MB 6.4 MB/s eta 0:00:01
     -------------------- ------------------- 3.0/5.7 MB 6.3 MB/s eta 0:00:01
     ---------------------- ----------------- 3.3/5.7 MB 6.4 MB/s eta 0:00:01
     ------------------------- -------------- 3.7/5.7 MB 6.3 MB/s eta 0:00:01
     ---------------------------- ----------- 4.1/5.7 MB 6.5 MB/s eta 0:00:01
     ------------------------------ --------- 4.4/5.7 MB 6.5 MB/s eta 0


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
# ローカルにおいている為不要
# from google.colab import drive
# drive.mount("/content/drive/")

## データセットの読み込み
Googleドライブに保存されている、ニュースのデータセットを読み込みます。

In [2]:
import glob  # ファイルの取得に使用
import os

path = "./text/"  # フォルダの場所を指定

dir_files = os.listdir(path=path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(path, f))]  # ディレクトリ一覧

text_label_data = []  # 文章とラベルのセット
dir_count = 0  # ディレクトリ数のカウント
file_count= 0  # ファイル数のカウント

# ディレクトリ内のtxtファイルを取得
for i in range(len(dirs)):
    dir = dirs[i]
    files = glob.glob(path + dir + "/*.txt")  # ファイルの一覧
    dir_count += 1

    for file in files:
        # licence.txtは不要なので無視
        if os.path.basename(file) == "LICENSE.txt":
            continue

        with open(file, "r", encoding='utf-8') as f:
            # 指定の行だけ読み込み（先頭の2行が不要なので除去）
            text = f.readlines()[3:]
            # 結合
            text = "".join(text)
            # 改行やタブを除去
            text = text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))
            # 本文とラベルをリストに格納
            text_label_data.append([text, i])

        file_count += 1
        print("\rfiles: " + str(file_count) + "dirs: " + str(dir_count), end="")

files: 4451dirs: 9

## データの保存
データを訓練データとテストデータに分割し、csvファイルとしてGoogle Driveに保存します。

In [4]:
import csv
from sklearn.model_selection import train_test_split

news_train, news_test =  train_test_split(text_label_data, shuffle=True)  # 訓練用とテスト用に分割
news_path = "./new_text"

with open(news_path+"news_train.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(news_train)

with open(news_path+"news_test.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(news_test)

## モデルとTokenizerの読み込み
日本語の事前学習済みモデルと、これと紐づいたTokenizerを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer
# num_label=9は9個に分類するよーってこと
sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=9)
sc_model.cuda()
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

## データセットの読み込み
保存されたニュースのデータを読み込みます。

In [ ]:
from datasets import load_dataset

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

news_path = "/content/drive/My Drive/bert_nlp/section_5/"

train_data = load_dataset("csv", data_files=news_path+"news_train.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

test_data = load_dataset("csv", data_files=news_path+"news_test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

## 評価用の関数
`sklearn.metrics`を使用し、モデルを評価するための関数を定義します。  


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

## Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerの設定を行います。
https://huggingface.co/transformers/main_classes/trainer.html   
https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments  

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    logging_dir = "./logs",
    evaluation_strategy = "steps"
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data,
)

## モデルの訓練
設定に基づきファインチューニングを行います。


In [ ]:
trainer.train()

## モデルの評価
Trainerの`evaluate()`メソッドによりモデルを評価します。

In [ ]:
trainer.evaluate()

## TensorBoardによる結果の表示
TensorBoardを使って、logsフォルダに格納された学習過程を表示します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルの保存
訓練済みのモデルを保存します。

In [ ]:
news_path = "/content/drive/My Drive/bert_nlp/section_5/"

sc_model.save_pretrained(news_path)
tokenizer.save_pretrained(news_path)

## モデルの読み込み
保存済みのモデルを読み込みます。

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(news_path)
loaded_model.cuda()
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(news_path)

## 日本語ニュースの分類
読み込んだモデルを使ってニュースを分類します。

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch

category = "movie-enter"
sample_path = "/content/drive/My Drive/bert_nlp/section_5/text/"  # フォルダの場所を指定
files = glob.glob(sample_path + category + "/*.txt")  # ファイルの一覧
file = files[12]  # 適当なニュース

dir_files = os.listdir(path=sample_path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(sample_path, f))]  # ディレクトリ一覧

with open(file, "r") as f:
    sample_text = f.readlines()[3:]
    sample_text = "".join(sample_text)
    sample_text = sample_text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))

print(sample_text)

max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換

x = word_tensor.cuda()  # GPU対応
y = loaded_model(x)  # 予測
pred = y[0].argmax(-1)  # 最大値のインデックス
print("result:", dirs[pred])